## Fine-Tune Stage 2

### Linux

### Debug Version

In [ ]:
# =======================
# Stage 2: Multi-task Fine-tuning (Sentiment + Sarcasm) [DEBUG MODE READY]
#   - Reuse Stage-1 LoRA + trained 3-way sentiment head
#   - Add a new 2-way sarcasm head
#   - Loss = CE_sarc + λ * CE_sent (masked by task)
#   - Save artifacts to stage2/stage2Result/{sent_result,sarc_result,output}
# =======================

import os, json, random, numpy as np, pandas as pd, torch
from pathlib import Path

# ---------- Basic env ----------
os.environ.setdefault("PYTORCH_ENABLE_MPS_FALLBACK", "0")
os.environ.setdefault("CUDA_VISIBLE_DEVICES", "0")
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")
torch.backends.cuda.matmul.allow_tf32 = True
assert torch.cuda.is_available(), "CUDA not found."

SEED = 47
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# ---------- Paths ----------
PROJECT_ROOT   = Path("/media/veracrypt1/DS5500/project")
SENT_CSV       = PROJECT_ROOT / "data" / "processed" / "sentiment_df.csv"
SARC_CSV       = PROJECT_ROOT / "data" / "processed" / "sarcasm_df.csv"
# ✅ Reuse Stage-1 artifacts saved under stage1Result (as you required)
STAGE1_DIR     = PROJECT_ROOT / "stage1" / "stage1Result" / "output"
STAGE1_ADAPTER = STAGE1_DIR / "adapter"
STAGE1_HEAD_PT = STAGE1_DIR / "sent_head.pt"

STAGE2_DIR     = PROJECT_ROOT / "stage2"
STAGE2_WORK    = STAGE2_DIR / "stage2_multitask"     # working dir for Trainer outputs
STAGE2_RESULT  = STAGE2_DIR / "stage2Result"         # result tree required by you
SENT_RESULT_DIR = STAGE2_RESULT / "sent_result"
SARC_RESULT_DIR = STAGE2_RESULT / "sarc_result"
OUTPUT_DIR      = STAGE2_RESULT / "output"
for d in [STAGE2_WORK, SENT_RESULT_DIR, SARC_RESULT_DIR, OUTPUT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
print("Device: cuda | GPU:", torch.cuda.get_device_name(0),
      "| VRAM (GB):", round(torch.cuda.get_device_properties(0).total_memory/1024**3,2))

# ---------- Debug switch (sample tiny subsets to fix pipeline first) ----------
DEBUG_SMALL = True
SARC_DEBUG_N = 1000
SENT_DEBUG_N = 1000
VAL_DEBUG_N  = 300   # per task for faster debug

# ---------- Read data ----------
def norm_split(s):
    s = str(s).strip().lower()
    if s in {"val","valid","validation","dev"}: return "validation"
    if s in {"train","training"}: return "train"
    if s in {"test","testing"}:  return "test"
    return s

# Sentiment
sent_df_all = pd.read_csv(SENT_CSV)
assert {"text","sentiment","split"}.issubset(sent_df_all.columns)
sent_df_all["split_norm"] = sent_df_all["split"].map(norm_split)
sent_train_df = sent_df_all[sent_df_all["split_norm"]=="train"][["text","sentiment"]].rename(columns={"sentiment":"label_sent"}).reset_index(drop=True)
sent_val_df   = sent_df_all[sent_df_all["split_norm"]=="validation"][["text","sentiment"]].rename(columns={"sentiment":"label_sent"}).reset_index(drop=True)
sent_train_df["label_sent"] = sent_train_df["label_sent"].astype(int)
sent_val_df["label_sent"]   = sent_val_df["label_sent"].astype(int)

# Sarcasm
sarc_df_all = pd.read_csv(SARC_CSV)
assert "text" in sarc_df_all.columns and "split" in sarc_df_all.columns
if "sarcasm" in sarc_df_all.columns:
    sarc_df_all = sarc_df_all.rename(columns={"sarcasm":"label_sarc"})
elif "label" in sarc_df_all.columns:
    sarc_df_all = sarc_df_all.rename(columns={"label":"label_sarc"})
else:
    raise ValueError("Sarcasm csv must have 'sarcasm' or 'label' column.")
sarc_df_all["split_norm"] = sarc_df_all["split"].map(norm_split)
sarc_train_df = sarc_df_all[sarc_df_all["split_norm"]=="train"][["text","label_sarc"]].reset_index(drop=True)
sarc_val_df   = sarc_df_all[sarc_df_all["split_norm"]=="validation"][["text","label_sarc"]].reset_index(drop=True)
sarc_train_df["label_sarc"] = sarc_train_df["label_sarc"].astype(int)
sarc_val_df["label_sarc"]   = sarc_val_df["label_sarc"].astype(int)

# ---------- Build replay (sentiment) + sarcasm multi-task train ----------
LAMBDA_SENT  = 0.5
REPLAY_RATIO = 1.0

replay_target = int(REPLAY_RATIO * len(sarc_train_df))
g = sent_train_df.groupby("label_sent", group_keys=False)
per_cls = max(1, replay_target // max(1, g.size().shape[0]))
replay_sent_df = g.apply(lambda x: x.sample(n=min(per_cls, len(x)), random_state=SEED),
                         include_groups=False).reset_index(drop=True)
if len(replay_sent_df) < replay_target and len(sent_train_df) > len(replay_sent_df):
    need = replay_target - len(replay_sent_df)
    replay_sent_df = pd.concat([replay_sent_df, sent_train_df.sample(n=need, random_state=SEED)], ignore_index=True)
replay_sent_df = replay_sent_df.sample(frac=1.0, random_state=SEED).reset_index(drop=Trudddddde)

# ---------- DEBUG subset sampling ----------
if DEBUG_SMALL:
    sarc_train_df = sarc_train_df.sample(n=min(SARC_DEBUG_N, len(sarc_train_df)), random_state=SEED).reset_index(drop=True)
    replay_sent_df = replay_sent_df.sample(n=min(SENT_DEBUG_N, len(replay_sent_df)), random_state=SEED).reset_index(drop=True)
    sent_val_df = sent_val_df.sample(n=min(VAL_DEBUG_N, len(sent_val_df)), random_state=SEED).reset_index(drop=True)
    sarc_val_df = sarc_val_df.sample(n=min(VAL_DEBUG_N, len(sarc_val_df)), random_state=SEED).reset_index(drop=True)

mt_train_df = pd.DataFrame({
    "text": pd.concat([sarc_train_df["text"], replay_sent_df["text"]], ignore_index=True),
})
mt_train_df["label_sarc"] = pd.concat([sarc_train_df["label_sarc"], pd.Series([-100]*len(replay_sent_df))], ignore_index=True)
mt_train_df["label_sent"] = pd.concat([pd.Series([-100]*len(sarc_train_df)), replay_sent_df["label_sent"]], ignore_index=True)
mt_train_df["task_id"]    = (mt_train_df["label_sarc"] != -100).astype(int)  # 1=sarc, 0=sent

both_mask = (mt_train_df["label_sent"]==-100) & (mt_train_df["label_sarc"]==-100)
assert not both_mask.any(), f"Bad rows with both labels -100: {both_mask.sum()}"

# Eval sets: each single-task with 'labels' column for HF
sent_val_df = sent_val_df[["text","label_sent"]].reset_index(drop=True)
sarc_val_df = sarc_val_df[["text","label_sarc"]].reset_index(drop=True)

print(f"[REPLAY] sentiment sampled = {len(replay_sent_df)}")
print(f"[MT-TRAIN] total={len(mt_train_df)} | frac sarcasm={mt_train_df['task_id'].mean():.3f}")
print(f"[SENT] train_replay={len(replay_sent_df)}, val={len(sent_val_df)}")
print(f"[SARC] train={len(sarc_train_df)}, val={len(sarc_val_df)}")

# ---------- Tokenization ----------
from datasets import Dataset
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tok.padding_side = "right"; tok.truncation_side = "right"
if tok.pad_token_id is None and tok.eos_token_id is not None:
    tok.pad_token = tok.eos_token

MAX_LEN = 128

def tok_mt_train(batch):
    enc = tok(batch["text"], truncation=True, padding="max_length", max_length=MAX_LEN)
    def nz(x):  # NaN -> -100
        return -100 if (x is None or (isinstance(x, float) and np.isnan(x))) else int(x)
    enc["labels_sent"] = [nz(x) for x in batch["label_sent"]]
    enc["labels_sarc"] = [nz(x) for x in batch["label_sarc"]]
    enc["task_id"]     = [int(x) for x in batch["task_id"]]
    return enc

def tok_eval_sent(batch):
    enc = tok(batch["text"], truncation=True, padding="max_length", max_length=MAX_LEN)
    enc["labels"]  = [int(x) for x in batch["label_sent"]]
    enc["task_id"] = [0]*len(batch["label_sent"])
    return enc

def tok_eval_sarc(batch):
    enc = tok(batch["text"], truncation=True, padding="max_length", max_length=MAX_LEN)
    enc["labels"]  = [int(x) for x in batch["label_sarc"]]
    enc["task_id"] = [1]*len(batch["label_sarc"])
    return enc

mt_train_tok = Dataset.from_pandas(mt_train_df).map(
    tok_mt_train, batched=True, remove_columns=list(mt_train_df.columns)
).with_format("torch")
sent_val_tok  = Dataset.from_pandas(sent_val_df).map(
    tok_eval_sent, batched=True, remove_columns=list(sent_val_df.columns)
).with_format("torch")
sarc_val_tok  = Dataset.from_pandas(sarc_val_df).map(
    tok_eval_sarc, batched=True, remove_columns=list(sarc_val_df.columns)
).with_format("torch")

# ---------- Load Stage-1 base + LoRA + trained 3-way head ----------
from transformers import AutoModelForSequenceClassification
from peft import PeftModel
from torch import nn

use_bf16    = torch.cuda.is_bf16_supported()
torch_dtype = torch.bfloat16 if use_bf16 else torch.float16

# base CLS model (3-way) + LoRA adapter
base_seqcls = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    problem_type="single_label_classification",
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
).to("cuda")

# attach Stage-1 LoRA adapter
base_seqcls = PeftModel.from_pretrained(base_seqcls, STAGE1_ADAPTER)

# ✅ load the trained Stage-1 3-way head
assert STAGE1_HEAD_PT.exists(), f"Missing Stage-1 head: {STAGE1_HEAD_PT}"
head_state = torch.load(STAGE1_HEAD_PT, map_location="cpu")
base_seqcls.score.load_state_dict(head_state)

# ---------- Multi-task wrapper ----------
class MultiTaskSentSarc(nn.Module):
    """
    Reuse Stage-1 peft_seqcls (with a trained 3-way head) + add a new 2-way sarcasm head.
    Masked loss: CE over available labels, loss = CE_sarc + lambda * CE_sent.
    """
    def __init__(self, peft_seqcls_model: nn.Module, pad_token_id: int, lambda_sent: float = 0.5):
        super().__init__()
        self.peft_seqcls = peft_seqcls_model
        self.pad_token_id = pad_token_id
        self.lambda_sent = lambda_sent
        self.ce = nn.CrossEntropyLoss(reduction="none")
        # dtype/device alignment for the new head
        sample_param = next(self.peft_seqcls.parameters())
        hidden = self.peft_seqcls.config.hidden_size
        self.sarc_head = nn.Linear(hidden, 2, device=sample_param.device, dtype=sample_param.dtype)
        # ensure hidden states are returned
        self.peft_seqcls.config.output_hidden_states = True

    def _pool_last_nonpad(self, hidden_states, attention_mask):
        if attention_mask is None:
            return hidden_states[:, -1, :]
        idx = attention_mask.long().sum(dim=1) - 1
        idx = torch.clamp(idx, min=0)
        return hidden_states[torch.arange(hidden_states.size(0), device=hidden_states.device), idx]

    def forward(self, input_ids=None, attention_mask=None,
                labels_sent=None, labels_sarc=None, task_id=None, **kwargs):
        # unwrap backbone
        if hasattr(self.peft_seqcls, "get_base_model"):
            backbone = self.peft_seqcls.get_base_model().model
        else:
            backbone = getattr(getattr(self.peft_seqcls, "base_model", self.peft_seqcls), "model", self.peft_seqcls)

        out = backbone(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True,
        )
        hidden = out.last_hidden_state if hasattr(out, "last_hidden_state") else out.hidden_states[-1]
        pooled = self._pool_last_nonpad(hidden, attention_mask)

        logits_sent = self.peft_seqcls.score(pooled)             # [B,3]
        logits_sarc = self.sarc_head(pooled)                     # [B,2]

        # masked multi-task loss
        loss = None
        if labels_sent is not None and (labels_sent != -100).any():
            m = (labels_sent != -100)
            loss_sent = self.ce(logits_sent[m], labels_sent[m]).mean()
            loss = (self.lambda_sent * loss_sent) if loss is None else (loss + self.lambda_sent * loss_sent)
        if labels_sarc is not None and (labels_sarc != -100).any():
            m = (labels_sarc != -100)
            loss_sarc = self.ce(logits_sarc[m], labels_sarc[m]).mean()
            loss = loss_sarc if loss is None else (loss + loss_sarc)
        if loss is None:
            loss = (logits_sent.mean()*0) + (logits_sarc.mean()*0)

        # for eval convenience: if the whole batch is sarcasm, switch logits to 2-way
        logits = logits_sent
        if task_id is not None and torch.all(task_id == 1):
            logits = logits_sarc

        return {"loss": loss, "logits": logits,
                "logits_sent": logits_sent, "logits_sarc": logits_sarc}

mt_model = MultiTaskSentSarc(base_seqcls, pad_token_id=tok.pad_token_id, lambda_sent=LAMBDA_SENT).to("cuda")

# ---------- Custom Trainer ----------
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

class MTTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # pop our custom labels; keep 'labels' for eval only
        labels_sent = inputs.pop("labels_sent", None)
        labels_sarc = inputs.pop("labels_sarc", None)
        outputs = model(**inputs, labels_sent=labels_sent, labels_sarc=labels_sarc)
        loss = outputs.get("loss", None)
        if loss is None:
            ce = torch.nn.CrossEntropyLoss(reduction="none")
            ls = lr = 0.0; count = 0
            if labels_sarc is not None and (labels_sarc != -100).any():
                m = (labels_sarc != -100)
                lr = ce(outputs["logits_sarc"][m], labels_sarc[m]).mean()
                loss = lr; count += 1
            if labels_sent is not None and (labels_sent != -100).any():
                m = (labels_sent != -100)
                ls = ce(outputs["logits_sent"][m], labels_sent[m]).mean()
                loss = (loss if count else 0) + getattr(model, "lambda_sent", 0.5) * ls
            if loss is None:
                loss = outputs["logits"].mean()*0
        return (loss, outputs) if return_outputs else loss

    # Ensure metrics see a single (N, C) logits tensor
    def preprocess_logits_for_metrics(self, logits, labels):
        if isinstance(logits, (tuple, list)):
            return logits[0]
        return logits

# Metrics for single-task eval sets (labels = 0/1/2 or 0/1)
def basic_metrics(p):
    preds = p.predictions[0] if isinstance(p.predictions, (tuple,list)) else p.predictions
    pred = np.argmax(preds, axis=-1)
    return {"accuracy": float(accuracy_score(p.label_ids, pred)),
            "macro_f1": float(f1_score(p.label_ids, pred, average="macro"))}

BATCH_SIZE = 8
GRAD_ACCUM = 4

mt_args = TrainingArguments(
    output_dir=str(STAGE2_WORK),
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=2e-5,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    warmup_steps=50 if DEBUG_SMALL else 100,
    num_train_epochs=1 if DEBUG_SMALL else 4,
    evaluation_strategy="epoch",
    save_strategy="no",
    load_best_model_at_end=False,
    logging_steps=20 if DEBUG_SMALL else 50,
    bf16=torch.cuda.is_bf16_supported(),
    fp16=not torch.cuda.is_bf16_supported(),
    dataloader_pin_memory=True,
    dataloader_num_workers=8,
    optim="adamw_torch",
    report_to="none",
    seed=SEED,
    # ✅ CRITICAL: keep ONLY "labels" here; do not include labels_sent/labels_sarc
    label_names=["labels"],
    save_safetensors=False,
)

mt_trainer = MTTrainer(
    model=mt_model,
    args=mt_args,
    train_dataset=mt_train_tok,
    eval_dataset=sent_val_tok,     # default eval = sentiment; we will evaluate both tasks below
    tokenizer=tok,
    compute_metrics=basic_metrics,
)

# ---------- Train ----------
train_result = mt_trainer.train()
print("\n[STAGE-2] train done (debug mode =", DEBUG_SMALL, ").")

# ---------- Eval helper (safe against pred.label_ids=None) ----------
def eval_and_save(name, eval_ds, out_dir, n_classes, id2label):
    """
    - Uses trainer.evaluate for loss + metrics (compute_metrics uses label_ids provided internally).
    - Uses trainer.predict for per-sample outputs, but y_true is read directly from eval_ds["labels"] to be robust.
    - Saves confusion matrix, classification report, per-sample CSV, raw logits.npy.
    """
    out_dir.mkdir(parents=True, exist_ok=True)

    # 1) evaluate() – gives loss, and calls compute_metrics (which uses label_ids extracted by Trainer)
    r = mt_trainer.evaluate(eval_dataset=eval_ds, metric_key_prefix=name)
    with open(out_dir / "metrics_eval.json", "w") as f:
        json.dump({k: float(v) if isinstance(v, (int,float,np.floating)) else v for k,v in r.items()}, f, indent=2)

    # 2) predict() – robustly get logits and y_true
    pred = mt_trainer.predict(eval_ds)
    logits = pred.predictions[0] if isinstance(pred.predictions, (tuple, list)) else pred.predictions
    y_pred = np.argmax(logits, axis=-1)

    # read gold labels directly from dataset (robust to HF quirks)
    y_true = np.array(eval_ds["labels"], dtype=int)

    # metrics again (manual)
    acc = accuracy_score(y_true, y_pred)
    mf1 = f1_score(y_true, y_pred, average="macro")
    cm  = confusion_matrix(y_true, y_pred, labels=list(range(n_classes)))

    # save confusion matrix
    cm_df = pd.DataFrame(cm, index=[id2label[i] for i in range(n_classes)],
                            columns=[id2label[i] for i in range(n_classes)])
    cm_df.to_csv(out_dir / "confusion_matrix.csv", index=True)

    # save classification report
    report_str = classification_report(y_true, y_pred,
                                       target_names=[id2label[i] for i in range(n_classes)],
                                       digits=4)
    with open(out_dir / "classification_report.txt", "w") as f:
        f.write(report_str)

    # save per-sample with text
    # need original text side-by-side:
    if name == "sentiment":
        src_df = sent_val_df.copy()
        true_map_col = "label_sent"
    else:
        src_df = sarc_val_df.copy()
        true_map_col = "label_sarc"

    per_sample = pd.DataFrame({
        "pred_label": y_pred,
        "pred_name":  [id2label[int(i)] for i in y_pred],
        "true_label": y_true,
        "true_name":  [id2label[int(i)] for i in y_true],
    })
    # align lengths if sampling caused minor length drift
    L = min(len(src_df), len(per_sample))
    src_df = src_df.iloc[:L].reset_index(drop=True)
    per_sample = per_sample.iloc[:L].reset_index(drop=True)
    merged = pd.concat([src_df, per_sample], axis=1)
    merged.to_csv(out_dir / "val_predictions_with_text.csv", index=False)

    # save raw logits
    np.save(out_dir / "val_logits.npy", logits)

    # console print
    print(f"\n=== {name.upper()} | validation | N={len(y_true)} ===")
    print(f"Accuracy: {acc:.4f} | Macro-F1: {mf1:.4f}")
    print(report_str)

# ---------- Evaluate (two tasks) ----------
id2label_sent = {0:"negative", 1:"neutral", 2:"positive"}
id2label_sarc = {0:"non_sarcastic", 1:"sarcastic"}

eval_and_save("sentiment", sent_val_tok, SENT_RESULT_DIR, n_classes=3, id2label=id2label_sent)
eval_and_save("sarcasm",   sarc_val_tok, SARC_RESULT_DIR, n_classes=2, id2label=id2label_sarc)

# ---------- Save adapters + heads + configs ----------
# Save peft adapter (LoRA) as used in Stage-2 (same adapter, but we snapshot to Stage-2 result tree)
ADAPTER_DIR = OUTPUT_DIR / "adapter"
ADAPTER_DIR.mkdir(parents=True, exist_ok=True)
# base_seqcls is inside mt_model.peft_seqcls; it is a PeftModel-wrapped SeqCls
mt_model.peft_seqcls.save_pretrained(ADAPTER_DIR)
tok.save_pretrained(ADAPTER_DIR)

# Save both heads (.pt)
torch.save(mt_model.peft_seqcls.score.state_dict(), OUTPUT_DIR / "sent_head.pt")
torch.save(mt_model.sarc_head.state_dict(),        OUTPUT_DIR / "sarc_head.pt")

# Save run meta
with open(OUTPUT_DIR / "run_meta_stage2.json", "w") as f:
    json.dump({
        "seed": SEED,
        "debug_mode": DEBUG_SMALL,
        "lambda_sent": LAMBDA_SENT,
        "replay_ratio": REPLAY_RATIO,
        "train_sizes": {
            "sarc_train": int(len(sarc_train_df)),
            "sent_replay_train": int(len(replay_sent_df)),
            "mt_train_total": int(len(mt_train_df)),
        },
        "val_sizes": {
            "sent_val": int(len(sent_val_df)),
            "sarc_val": int(len(sarc_val_df)),
        },
        "id2label_sent": id2label_sent,
        "id2label_sarc": id2label_sarc,
        "stage1_adapter_dir": str(STAGE1_ADAPTER),
        "stage1_head_pt": str(STAGE1_HEAD_PT)
    }, f, indent=2)

print("\n== Saved Stage-2 artifacts ==")
print("Adapter dir:", ADAPTER_DIR)
print("Sentiment head:", OUTPUT_DIR / "sent_head.pt")
print("Sarcasm head:",   OUTPUT_DIR / "sarc_head.pt")
print("Sentiment results @", SENT_RESULT_DIR)
print("Sarcasm   results @", SARC_RESULT_DIR)


Device: cuda | GPU: NVIDIA GeForce RTX 4090 | VRAM (GB): 23.64
[REPLAY] sentiment sampled = 1000
[MT-TRAIN] total=2000 | frac sarcasm=0.500
[SENT] train_replay=1000, val=300
[SARC] train=1000, val=300


/home/exx/venvs/llamaFT_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 12.40it/s]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.1-8B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/exx/venvs/llamaFT_env/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 32%|███▏      | 20/62 [00:11<00:23,  1.78it/s]

{'loss': 0.7835, 'grad_norm': 71.5, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.32}


 65%|██████▍   | 40/62 [00:22<00:12,  1.80it/s]

{'loss': 0.7836, 'grad_norm': 67.5, 'learning_rate': 1.6000000000000003e-05, 'epoch': 0.64}


...

{'eval_loss': 0.0, 'eval_accuracy': 0.81, 'eval_macro_f1': 0.7866685629437175, 'eval_runtime': 5.04, 'eval_samples_per_second': 59.523, 'eval_steps_per_second': 7.54, 'epoch': 0.99}
{'train_runtime': 40.3986, 'train_samples_per_second': 49.507, 'train_steps_per_second': 1.535, 'train_loss': 0.7704854165354083, 'epoch': 0.99}

[STAGE-2] train done (debug mode = True ).



100%|██████████| 38/38 [00:04<00:00,  8.05it/s]


=== SENTIMENT | validation | N=300 ===
Accuracy: 0.8100 | Macro-F1: 0.7867
              precision    recall  f1-score   support

    negative     0.8586    0.8173    0.8374       104
     neutral     0.5897    0.6765    0.6301        68
    positive     0.9106    0.8750    0.8924       128

    accuracy                         0.8100       300
   macro avg     0.7863    0.7896    0.7867       300
weighted avg     0.8198    0.8100    0.8139       300




100%|██████████| 38/38 [00:04<00:00,  7.99it/s]



=== SARCASM | validation | N=300 ===
Accuracy: 0.3833 | Macro-F1: 0.3650
               precision    recall  f1-score   support

non_sarcastic     0.3902    0.1916    0.2570       167
    sarcastic     0.3807    0.6241    0.4729       133

     accuracy                         0.3833       300
    macro avg     0.3855    0.4078    0.3650       300
 weighted avg     0.3860    0.3833    0.3527       300


== Saved Stage-2 artifacts ==
Adapter dir: /media/veracrypt1/DS5500/project/stage2/stage2Result/output/adapter
Sentiment head: /media/veracrypt1/DS5500/project/stage2/stage2Result/output/sent_head.pt
Sarcasm head: /media/veracrypt1/DS5500/project/stage2/stage2Result/output/sarc_head.pt
Sentiment results @ /media/veracrypt1/DS5500/project/stage2/stage2Result/sent_result
Sarcasm   results @ /media/veracrypt1/DS5500/project/stage2/stage2Result/sarc_result


### Formal Version

In [1]:
# =======================
# Stage 2: Multi-task Fine-tuning (Sentiment + Sarcasm) [PRODUCTION]
#   - Reuse Stage-1 LoRA + trained 3-way sentiment head
#   - Add a new 2-way sarcasm head
#   - Loss = CE_sarc + λ * CE_sent  (masked by task)
#   - Save artifacts to:
#       /stage2/stage2Result/{sent_result, sarc_result, output}
#   - All comments in English
# =======================

import os, json, random, numpy as np, pandas as pd, torch
from pathlib import Path

# ---------- Basic env ----------
os.environ.setdefault("PYTORCH_ENABLE_MPS_FALLBACK", "0")
os.environ.setdefault("CUDA_VISIBLE_DEVICES", "0")
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")
torch.backends.cuda.matmul.allow_tf32 = True
assert torch.cuda.is_available(), "CUDA not found."

SEED = 47
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# ---------- Paths ----------
PROJECT_ROOT    = Path("/media/veracrypt1/DS5500/project")
SENT_CSV        = PROJECT_ROOT / "data" / "processed" / "sentiment_df.csv"
SARC_CSV        = PROJECT_ROOT / "data" / "processed" / "sarcasm_df.csv"

# Reuse Stage-1 artifacts saved as required
STAGE1_DIR      = PROJECT_ROOT / "stage1" / "stage1Result" / "output"
STAGE1_ADAPTER  = STAGE1_DIR / "adapter"
STAGE1_HEAD_PT  = STAGE1_DIR / "sent_head.pt"

STAGE2_DIR      = PROJECT_ROOT / "stage2"
STAGE2_WORK     = STAGE2_DIR / "stage2_multitask"     # Trainer working dir
STAGE2_RESULT   = STAGE2_DIR / "stage2Result"         # Final results tree
SENT_RESULT_DIR = STAGE2_RESULT / "sent_result"
SARC_RESULT_DIR = STAGE2_RESULT / "sarc_result"
OUTPUT_DIR      = STAGE2_RESULT / "output"
for d in [STAGE2_WORK, SENT_RESULT_DIR, SARC_RESULT_DIR, OUTPUT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
print("Device: cuda | GPU:", torch.cuda.get_device_name(0),
      "| VRAM (GB):", round(torch.cuda.get_device_properties(0).total_memory/1024**3,2))

# ---------- PRODUCTION switch ----------
DEBUG_SMALL = False           # <- production run (set True for quick debug)
SARC_DEBUG_N = 1000           # used only if DEBUG_SMALL=True
SENT_DEBUG_N = 1000
VAL_DEBUG_N  = 300

# ---------- Read & split ----------
def norm_split(s):
    s = str(s).strip().lower()
    if s in {"val","valid","validation","dev"}: return "validation"
    if s in {"train","training"}: return "train"
    if s in {"test","testing"}:  return "test"
    return s

# Sentiment
sent_df_all = pd.read_csv(SENT_CSV)
assert {"text","sentiment","split"}.issubset(sent_df_all.columns)
sent_df_all["split_norm"] = sent_df_all["split"].map(norm_split)
sent_train_df = sent_df_all[sent_df_all["split_norm"]=="train"][["text","sentiment"]].rename(
    columns={"sentiment":"label_sent"}).reset_index(drop=True)
sent_val_df   = sent_df_all[sent_df_all["split_norm"]=="validation"][["text","sentiment"]].rename(
    columns={"sentiment":"label_sent"}).reset_index(drop=True)
sent_train_df["label_sent"] = sent_train_df["label_sent"].astype(int)
sent_val_df["label_sent"]   = sent_val_df["label_sent"].astype(int)

# Sarcasm
sarc_df_all = pd.read_csv(SARC_CSV)
assert "text" in sarc_df_all.columns and "split" in sarc_df_all.columns
if "sarcasm" in sarc_df_all.columns:
    sarc_df_all = sarc_df_all.rename(columns={"sarcasm":"label_sarc"})
elif "label" in sarc_df_all.columns:
    sarc_df_all = sarc_df_all.rename(columns={"label":"label_sarc"})
else:
    raise ValueError("Sarcasm csv must have 'sarcasm' or 'label' column.")
sarc_df_all["split_norm"] = sarc_df_all["split"].map(norm_split)
sarc_train_df = sarc_df_all[sarc_df_all["split_norm"]=="train"][["text","label_sarc"]].reset_index(drop=True)
sarc_val_df   = sarc_df_all[sarc_df_all["split_norm"]=="validation"][["text","label_sarc"]].reset_index(drop=True)
sarc_train_df["label_sarc"] = sarc_train_df["label_sarc"].astype(int)
sarc_val_df["label_sarc"]   = sarc_val_df["label_sarc"].astype(int)

# ---------- Build replay (sentiment) + sarcasm multi-task train ----------
LAMBDA_SENT  = 0.5     # loss = CE_sarc + λ * CE_sent
REPLAY_RATIO = 1.0     # sentiment replay size relative to sarcasm train size

replay_target = int(REPLAY_RATIO * len(sarc_train_df))
g = sent_train_df.groupby("label_sent", group_keys=False)
per_cls = max(1, replay_target // max(1, g.size().shape[0]))
# stratified-ish balanced sampling
replay_sent_df = g.apply(lambda x: x.sample(n=min(per_cls, len(x)), random_state=SEED),
                         include_groups=False).reset_index(drop=True)
if len(replay_sent_df) < replay_target and len(sent_train_df) > len(replay_sent_df):
    need = replay_target - len(replay_sent_df)
    replay_sent_df = pd.concat([replay_sent_df, sent_train_df.sample(n=need, random_state=SEED)],
                               ignore_index=True)
replay_sent_df = replay_sent_df.sample(frac=1.0, random_state=SEED).reset_index(drop=True)

# Optional debug subsampling
if DEBUG_SMALL:
    sarc_train_df  = sarc_train_df.sample(n=min(SARC_DEBUG_N, len(sarc_train_df)),
                                          random_state=SEED).reset_index(drop=True)
    replay_sent_df = replay_sent_df.sample(n=min(SENT_DEBUG_N, len(replay_sent_df)),
                                           random_state=SEED).reset_index(drop=True)
    sent_val_df    = sent_val_df.sample(n=min(VAL_DEBUG_N, len(sent_val_df)),
                                        random_state=SEED).reset_index(drop=True)
    sarc_val_df    = sarc_val_df.sample(n=min(VAL_DEBUG_N, len(sarc_val_df)),
                                        random_state=SEED).reset_index(drop=True)

# Merge into multi-task training dataframe
mt_train_df = pd.DataFrame({
    "text": pd.concat([sarc_train_df["text"], replay_sent_df["text"]], ignore_index=True),
})
mt_train_df["label_sarc"] = pd.concat(
    [sarc_train_df["label_sarc"], pd.Series([-100]*len(replay_sent_df))], ignore_index=True)
mt_train_df["label_sent"] = pd.concat(
    [pd.Series([-100]*len(sarc_train_df)), replay_sent_df["label_sent"]], ignore_index=True)
mt_train_df["task_id"]    = (mt_train_df["label_sarc"] != -100).astype(int)  # 1=sarc, 0=sent

both_mask = (mt_train_df["label_sent"]==-100) & (mt_train_df["label_sarc"]==-100)
assert not both_mask.any(), f"Bad rows with both labels -100: {both_mask.sum()}"

# Eval sets: each single-task with 'labels' column for HF Trainer
sent_val_df = sent_val_df[["text","label_sent"]].reset_index(drop=True)
sarc_val_df = sarc_val_df[["text","label_sarc"]].reset_index(drop=True)

print(f"[REPLAY] sentiment sampled = {len(replay_sent_df)}")
print(f"[MT-TRAIN] total={len(mt_train_df)} | frac sarcasm={mt_train_df['task_id'].mean():.3f}")
print(f"[SENT] train_replay={len(replay_sent_df)}, val={len(sent_val_df)}")
print(f"[SARC] train={len(sarc_train_df)}, val={len(sarc_val_df)}")

# ---------- Tokenization ----------
from datasets import Dataset
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tok.padding_side = "right"; tok.truncation_side = "right"
if tok.pad_token_id is None and tok.eos_token_id is not None:
    tok.pad_token = tok.eos_token

MAX_LEN = 128

def tok_mt_train(batch):
    enc = tok(batch["text"], truncation=True, padding="max_length", max_length=MAX_LEN)
    def nz(x):  # NaN -> -100
        return -100 if (x is None or (isinstance(x, float) and np.isnan(x))) else int(x)
    enc["labels_sent"] = [nz(x) for x in batch["label_sent"]]
    enc["labels_sarc"] = [nz(x) for x in batch["label_sarc"]]
    enc["task_id"]     = [int(x) for x in batch["task_id"]]
    return enc

def tok_eval_sent(batch):
    enc = tok(batch["text"], truncation=True, padding="max_length", max_length=MAX_LEN)
    enc["labels"]  = [int(x) for x in batch["label_sent"]]
    enc["task_id"] = [0]*len(batch["label_sent"])     # whole batch = sentiment
    return enc

def tok_eval_sarc(batch):
    enc = tok(batch["text"], truncation=True, padding="max_length", max_length=MAX_LEN)
    enc["labels"]  = [int(x) for x in batch["label_sarc"]]
    enc["task_id"] = [1]*len(batch["label_sarc"])     # whole batch = sarcasm
    return enc

mt_train_tok = Dataset.from_pandas(mt_train_df).map(
    tok_mt_train, batched=True, remove_columns=list(mt_train_df.columns)
).with_format("torch")
sent_val_tok  = Dataset.from_pandas(sent_val_df).map(
    tok_eval_sent, batched=True, remove_columns=list(sent_val_df.columns)
).with_format("torch")
sarc_val_tok  = Dataset.from_pandas(sarc_val_df).map(
    tok_eval_sarc, batched=True, remove_columns=list(sarc_val_df.columns)
).with_format("torch")

# ---------- Load Stage-1 base + LoRA + trained 3-way head ----------
from transformers import AutoModelForSequenceClassification
from peft import PeftModel
from torch import nn

use_bf16    = torch.cuda.is_bf16_supported()
torch_dtype = torch.bfloat16 if use_bf16 else torch.float16

base_seqcls = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    problem_type="single_label_classification",
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
).to("cuda")

# attach Stage-1 adapter
base_seqcls = PeftModel.from_pretrained(base_seqcls, STAGE1_ADAPTER)

# load Stage-1 trained 3-way head
assert STAGE1_HEAD_PT.exists(), f"Missing Stage-1 head: {STAGE1_HEAD_PT}"
head_state = torch.load(STAGE1_HEAD_PT, map_location="cpu")
base_seqcls.score.load_state_dict(head_state)

# ---------- Multi-task wrapper ----------
class MultiTaskSentSarc(nn.Module):
    """
    Reuse Stage-1 peft_seqcls (with trained 3-way head) + add a new 2-way sarcasm head.
    Masked loss: CE over available labels, loss = CE_sarc + lambda * CE_sent.
    For evaluation:
      - If the batch is all sarcasm (task_id=1), return 2-way logits
      - Otherwise return 3-way logits (default)
    """
    def __init__(self, peft_seqcls_model: nn.Module, pad_token_id: int, lambda_sent: float = 0.5):
        super().__init__()
        self.peft_seqcls = peft_seqcls_model
        self.pad_token_id = pad_token_id
        self.lambda_sent = lambda_sent
        self.ce = nn.CrossEntropyLoss(reduction="none")
        # dtype/device alignment for the new sarcasm head
        sample_param = next(self.peft_seqcls.parameters())
        hidden = self.peft_seqcls.config.hidden_size
        self.sarc_head = nn.Linear(hidden, 2, device=sample_param.device, dtype=sample_param.dtype)
        # ensure hidden states are returned
        self.peft_seqcls.config.output_hidden_states = True

    def _pool_last_nonpad(self, hidden_states, attention_mask):
        if attention_mask is None:
            return hidden_states[:, -1, :]
        idx = attention_mask.long().sum(dim=1) - 1
        idx = torch.clamp(idx, min=0)
        return hidden_states[torch.arange(hidden_states.size(0), device=hidden_states.device), idx]

    def forward(self, input_ids=None, attention_mask=None,
                labels_sent=None, labels_sarc=None, task_id=None, **kwargs):
        # unwrap backbone (transformer body)
        if hasattr(self.peft_seqcls, "get_base_model"):
            backbone = self.peft_seqcls.get_base_model().model
        else:
            backbone = getattr(getattr(self.peft_seqcls, "base_model", self.peft_seqcls), "model", self.peft_seqcls)

        out = backbone(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True,
        )
        hidden = out.last_hidden_state if hasattr(out, "last_hidden_state") else out.hidden_states[-1]
        pooled = self._pool_last_nonpad(hidden, attention_mask)

        logits_sent = self.peft_seqcls.score(pooled)   # [B,3]
        logits_sarc = self.sarc_head(pooled)           # [B,2]

        # masked multi-task loss
        loss = None
        if labels_sent is not None and (labels_sent != -100).any():
            m = (labels_sent != -100)
            loss_sent = self.ce(logits_sent[m], labels_sent[m]).mean()
            loss = (self.lambda_sent * loss_sent) if loss is None else (loss + self.lambda_sent * loss_sent)
        if labels_sarc is not None and (labels_sarc != -100).any():
            m = (labels_sarc != -100)
            loss_sarc = self.ce(logits_sarc[m], labels_sarc[m]).mean()
            loss = loss_sarc if loss is None else (loss + loss_sarc)
        if loss is None:
            loss = (logits_sent.mean()*0) + (logits_sarc.mean()*0)

        # return one logits tensor for HF predict/eval:
        logits = logits_sent
        if task_id is not None and torch.all(task_id == 1):
            logits = logits_sarc

        return {"loss": loss, "logits": logits,
                "logits_sent": logits_sent, "logits_sarc": logits_sarc}

mt_model = MultiTaskSentSarc(base_seqcls, pad_token_id=tok.pad_token_id, lambda_sent=LAMBDA_SENT).to("cuda")

# ---------- Custom Trainer ----------
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

class MTTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # pop custom labels; keep 'labels' (eval only)
        labels_sent = inputs.pop("labels_sent", None)
        labels_sarc = inputs.pop("labels_sarc", None)
        outputs = model(**inputs, labels_sent=labels_sent, labels_sarc=labels_sarc)
        loss = outputs.get("loss", None)
        if loss is None:
            ce = torch.nn.CrossEntropyLoss(reduction="none")
            ls = lr = 0.0; count = 0
            if labels_sarc is not None and (labels_sarc != -100).any():
                m = (labels_sarc != -100)
                lr = ce(outputs["logits_sarc"][m], labels_sarc[m]).mean()
                loss = lr; count += 1
            if labels_sent is not None and (labels_sent != -100).any():
                m = (labels_sent != -100)
                ls = ce(outputs["logits_sent"][m], labels_sent[m]).mean()
                loss = (loss if count else 0) + getattr(model, "lambda_sent", 0.5) * ls
            if loss is None:
                loss = outputs["logits"].mean()*0
        return (loss, outputs) if return_outputs else loss

    # Ensure metrics always see a single (N, C) logits tensor
    def preprocess_logits_for_metrics(self, logits, labels):
        if isinstance(logits, (tuple, list)):
            return logits[0]
        return logits

def basic_metrics(p):
    preds = p.predictions[0] if isinstance(p.predictions, (tuple,list)) else p.predictions
    pred = np.argmax(preds, axis=-1)
    return {
        "accuracy": float(accuracy_score(p.label_ids, pred)),
        "macro_f1": float(f1_score(p.label_ids, pred, average="macro"))
    }

BATCH_SIZE = 8
GRAD_ACCUM = 4

mt_args = TrainingArguments(
    output_dir=str(STAGE2_WORK),
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=2e-5,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    warmup_steps=100 if not DEBUG_SMALL else 50,
    num_train_epochs=4 if not DEBUG_SMALL else 1,
    evaluation_strategy="epoch",
    save_strategy="no",
    load_best_model_at_end=False,
    logging_steps=50 if not DEBUG_SMALL else 20,
    bf16=torch.cuda.is_bf16_supported(),
    fp16=not torch.cuda.is_bf16_supported(),
    dataloader_pin_memory=True,
    dataloader_num_workers=8,
    optim="adamw_torch",
    report_to="none",
    seed=SEED,
    # IMPORTANT: keep ONLY "labels" here; do not include labels_sent/labels_sarc
    label_names=["labels"],
    save_safetensors=False,
)

mt_trainer = MTTrainer(
    model=mt_model,
    args=mt_args,
    train_dataset=mt_train_tok,
    eval_dataset=sent_val_tok,     # default eval = sentiment; we'll evaluate both below
    tokenizer=tok,
    compute_metrics=basic_metrics,
)

# ---------- Train ----------
train_result = mt_trainer.train()
print("\n[STAGE-2] train done. DEBUG_SMALL =", DEBUG_SMALL)

# ---------- Eval helper (robust to HF quirks) ----------
def eval_and_save(name, eval_ds, out_dir, n_classes, id2label):
    """
    - Uses trainer.evaluate for loss/metrics.
    - Uses trainer.predict for per-sample outputs; gold labels read from eval_ds['labels'].
    - Saves confusion matrix / classification report / per-sample CSV / raw logits.npy.
    """
    out_dir.mkdir(parents=True, exist_ok=True)

    # 1) evaluate (loss + compute_metrics)
    r = mt_trainer.evaluate(eval_dataset=eval_ds, metric_key_prefix=name)
    with open(out_dir / "metrics_eval.json", "w") as f:
        json.dump({k: float(v) if isinstance(v, (int,float,np.floating)) else v for k,v in r.items()}, f, indent=2)

    # 2) predict (logits)
    pred = mt_trainer.predict(eval_ds)
    logits = pred.predictions[0] if isinstance(pred.predictions, (tuple, list)) else pred.predictions
    y_pred = np.argmax(logits, axis=-1)

    # read gold labels directly from dataset
    y_true = np.array(eval_ds["labels"], dtype=int)

    # aggregate metrics
    acc = accuracy_score(y_true, y_pred)
    mf1 = f1_score(y_true, y_pred, average="macro")
    cm  = confusion_matrix(y_true, y_pred, labels=list(range(n_classes)))

    # save confusion matrix
    cm_df = pd.DataFrame(cm, index=[id2label[i] for i in range(n_classes)],
                            columns=[id2label[i] for i in range(n_classes)])
    cm_df.to_csv(out_dir / "confusion_matrix.csv", index=True)

    # save classification report
    report_str = classification_report(
        y_true, y_pred, target_names=[id2label[i] for i in range(n_classes)], digits=4
    )
    with open(out_dir / "classification_report.txt", "w") as f:
        f.write(report_str)

    # save per-sample predictions with text
    if name == "sentiment":
        src_df = sent_val_df.copy()
    else:
        src_df = sarc_val_df.copy()

    per_sample = pd.DataFrame({
        "pred_label": y_pred,
        "pred_name":  [id2label[int(i)] for i in y_pred],
        "true_label": y_true,
        "true_name":  [id2label[int(i)] for i in y_true],
    })
    L = min(len(src_df), len(per_sample))
    src_df = src_df.iloc[:L].reset_index(drop=True)
    per_sample = per_sample.iloc[:L].reset_index(drop=True)
    merged = pd.concat([src_df, per_sample], axis=1)
    merged.to_csv(out_dir / "val_predictions_with_text.csv", index=False)

    # save raw logits
    np.save(out_dir / "val_logits.npy", logits)

    # console summary
    print(f"\n=== {name.upper()} | validation | N={len(y_true)} ===")
    print(f"Accuracy: {acc:.4f} | Macro-F1: {mf1:.4f}")
    print(report_str)

# ---------- Evaluate both tasks ----------
id2label_sent = {0:"negative", 1:"neutral", 2:"positive"}
id2label_sarc = {0:"non_sarcastic", 1:"sarcastic"}

eval_and_save("sentiment", sent_val_tok, SENT_RESULT_DIR, n_classes=3, id2label=id2label_sent)
eval_and_save("sarcasm",   sarc_val_tok, SARC_RESULT_DIR, n_classes=2, id2label=id2label_sarc)

# ---------- Save adapters + heads + configs ----------
# Save peft adapter (LoRA) to Stage-2 result tree (snapshot)
ADAPTER_DIR = OUTPUT_DIR / "adapter"
ADAPTER_DIR.mkdir(parents=True, exist_ok=True)
mt_model.peft_seqcls.save_pretrained(ADAPTER_DIR)   # LoRA adapter + peft config
tok.save_pretrained(ADAPTER_DIR)

# Save both heads (.pt)
torch.save(mt_model.peft_seqcls.score.state_dict(), OUTPUT_DIR / "sent_head.pt")
torch.save(mt_model.sarc_head.state_dict(),        OUTPUT_DIR / "sarc_head.pt")

# Save run meta
with open(OUTPUT_DIR / "run_meta_stage2.json", "w") as f:
    json.dump({
        "seed": SEED,
        "debug_mode": DEBUG_SMALL,
        "lambda_sent": LAMBDA_SENT,
        "replay_ratio": REPLAY_RATIO,
        "train_sizes": {
            "sarc_train": int(len(sarc_train_df)),
            "sent_replay_train": int(len(replay_sent_df)),
            "mt_train_total": int(len(mt_train_df)),
        },
        "val_sizes": {
            "sent_val": int(len(sent_val_df)),
            "sarc_val": int(len(sarc_val_df)),
        },
        "id2label_sent": id2label_sent,
        "id2label_sarc": id2label_sarc,
        "stage1_adapter_dir": str(STAGE1_ADAPTER),
        "stage1_head_pt": str(STAGE1_HEAD_PT)
    }, f, indent=2)

print("\n== Saved Stage-2 artifacts ==")
print("Adapter dir:", ADAPTER_DIR)
print("Sentiment head:", OUTPUT_DIR / "sent_head.pt")
print("Sarcasm head:",   OUTPUT_DIR / "sarc_head.pt")
print("Sentiment results @", SENT_RESULT_DIR)
print("Sarcasm   results @", SARC_RESULT_DIR)


Device: cuda | GPU: NVIDIA GeForce RTX 4090 | VRAM (GB): 23.64
[REPLAY] sentiment sampled = 11009
[MT-TRAIN] total=22018 | frac sarcasm=0.500
[SENT] train_replay=11009, val=8101
[SARC] train=11009, val=955


/home/exx/venvs/llamaFT_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 15.96it/s]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.1-8B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/exx/venvs/llamaFT_env/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  2%|▏         | 50/2752 [00:28<25:29,  1.77it/s]

{'loss': 0.8179, 'grad_norm': 27.5, 'learning_rate': 1e-05, 'epoch': 0.07}


  4%|▎         | 100/2752 [00:56<24:48,  1.78it/s]

{'loss': 0.7319, 'grad_norm': 30.125, 'learning_rate': 2e-05, 'epoch': 0.15}


...

{'eval_loss': 0.0, 'eval_accuracy': 0.7985433897049747, 'eval_macro_f1': 0.7867659779575287, 'eval_runtime': 130.4547, 'eval_samples_per_second': 62.098, 'eval_steps_per_second': 7.765, 'epoch': 4.0}
{'train_runtime': 2078.1359, 'train_samples_per_second': 42.38, 'train_steps_per_second': 1.324, 'train_loss': 0.6181378240242253, 'epoch': 4.0}

[STAGE-2] train done. DEBUG_SMALL = False



100%|██████████| 1013/1013 [02:09<00:00,  7.83it/s]



=== SENTIMENT | validation | N=8101 ===
Accuracy: 0.7985 | Macro-F1: 0.7868
              precision    recall  f1-score   support

    negative     0.8425    0.8296    0.8360      2740
     neutral     0.6092    0.7035    0.6530      2098
    positive     0.9128    0.8336    0.8714      3263

    accuracy                         0.7985      8101
   macro avg     0.7881    0.7889    0.7868      8101
weighted avg     0.8104    0.7985    0.8028      8101



100%|██████████| 120/120 [00:15<00:00,  7.94it/s]



=== SARCASM | validation | N=955 ===
Accuracy: 0.7717 | Macro-F1: 0.7642
               precision    recall  f1-score   support

non_sarcastic     0.7241    0.9098    0.8064       499
    sarcastic     0.8628    0.6206    0.7219       456

     accuracy                         0.7717       955
    macro avg     0.7934    0.7652    0.7642       955
 weighted avg     0.7903    0.7717    0.7661       955


== Saved Stage-2 artifacts ==
Adapter dir: /media/veracrypt1/DS5500/project/stage2/stage2Result/output/adapter
Sentiment head: /media/veracrypt1/DS5500/project/stage2/stage2Result/output/sent_head.pt
Sarcasm head: /media/veracrypt1/DS5500/project/stage2/stage2Result/output/sarc_head.pt
Sentiment results @ /media/veracrypt1/DS5500/project/stage2/stage2Result/sent_result
Sarcasm   results @ /media/veracrypt1/DS5500/project/stage2/stage2Result/sarc_result
